In [1]:
from Project_code import *

board = chess.Board()

pre_mate = chess.Board('rnbqkbnr/pppp1ppp/8/4p3/6P1/5P2/PPPPP2P/RNBQKBNR b KQkq - 0 2')

version()

V 0.2.10


In [ ]:
games = loadFromRepository()


In [36]:
def gamesToLibrary(o_games,minply = 0):

    games = copy.deepcopy(o_games)
    library = []
    for game in games:
        if game[0]=="1-0" or game[0] == 1:
            result = 1
        elif game[0]=="1/2-1/2" or game[0] == 0:
            result = 0
        else:
            result = -1
        while game[1].ply() > minply:
            
            library += [[copy.deepcopy(game[1]),result],]
            game[1].pop()
    return library

In [41]:
def gamesToLibrary2(o_games,minply = 0):

    games = copy.deepcopy(o_games)
    library = []
    for game in games:
        if game[0]=="1-0" or game[0] == 1:
            result = 1
        elif game[0]=="1/2-1/2" or game[0] == 0:
            result = 0
        else:
            result = -1
        while game[1].ply() > minply:
            
            library += [[chess.Board(game[1].fen()),result],]
            game[1].pop()
    return library

In [38]:
library = gamesToLibrary(games)

KeyboardInterrupt: 

In [42]:
library = gamesToLibrary2(games)

In [56]:
array = np.array(library).T

In [3]:
training_indexes = list(range(1,len(library[0])))
np.random.shuffle(training_indexes)
train_data = [library[0][training_indexes],library[1][training_indexes]]
train_pos =[]
for n,i in enumerate(train_data[0]):
    train_pos += [position_bitboard.getValues(i),]
train_lab = train_data[1]

TypeError: list indices must be integers or slices, not list

In [ ]:
train_pos = np.array(train_pos, dtype=np.float32)
train_lab = np.array(train_lab, dtype=np.float32)

In [ ]:
train_pos= train_pos[0:100]
train_lab=train_lab[0:100]

In [71]:
class Model(tf.keras.models.Sequential):
    def setName(self,name):
        self.name = name
        
    def getName(self):
        return self.name
    
    def save(self):
        if hasattr(self, 'name'):
            super().save(self.name + ".keras", *args, **kwargs)
        else:
            raise ValueError("Model name has not been set. Use setName() method to set the model name.")


In [73]:
first_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(12,8,8),name="flatten"),
    tf.keras.layers.Dense(100,activation="relu",name="layer2"),
    tf.keras.layers.Dense(1,activation="tanh",name="output")
])

first_model.compile(optimizer='adam',
              loss="mean_squared_error",
              metrics=['accuracy'])


In [66]:
first_model.save()

In [6]:
first_model.fit(x = train_pos,
          y = train_lab)

NameError: name 'train_pos' is not defined

In [10]:
model.predict(np.array([position_bitboard.getValues(board)]))

1/1 [==============================] - 0s 168ms/step


array([[0.32714742]], dtype=float32)

In [50]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 768)               590592    
                                                                 
 layer2 (Dense)              (None, 100)               76900     
                                                                 
 output (Dense)              (None, 1)                 101       
                                                                 
Total params: 667593 (2.55 MB)
Trainable params: 667593 (2.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [2]:
time.time()

1718674123.1001225

In [55]:
def ID_alphabeta_branch(board,n,eval,alpha=-2,beta=2): # iterative deepening alpha-beta prunning        
    def id_search(board,n,eval,alpha,beta):
        return classified_move_list #[([move1.1,move1.2],value1),([move2.1,move2.2],value2)]
    
    
    poss_moves = list(board.legal_moves)
    evals = []
    if len(poss_moves)>0: #Has possible moves -> evals each Node/Leaf    
        if n==1: #Leafs          
            if board.turn:
                value = -5
                for move in poss_moves:
                    t_board = chess.Board(board.fen())
                    t_board.push(move)
                    t_value = eval(t_board)
                    if t_value > value:
                        value = t_value
                        best_move = move
                    elif t_value == value:
                        r = random.random()
                        if r > 0.5:
                            value = t_value
                            best_move = move
            else:
                value = 5
                for move in poss_moves:
                    t_board = chess.Board(board.fen())
                    t_board.push(move)
                    t_value = eval(t_board)
                    if t_value < value:
                        value = t_value
                        best_move = move
                    elif t_value == value:
                        r = random.random()
                        if r > 0.5:
                            value = t_value
                            best_move = move
                                          
        else: #Branch Nodes
            if board.turn:
                value = -5
                for move in poss_moves:
                    t_board = chess.Board(board.fen())
                    t_board.push(move)
                    t_value = ab_branch(t_board,n-1,eval,alpha,beta)[1]
                    if t_value > value:
                        value = t_value
                        best_move = move
                    elif t_value == value:
                        r = random.random()
                        if r > 0.1:
                            value = t_value
                            best_move = move
                    if value > beta:
                        break
                    alpha = max(alpha,value)
                    
                         
            else:
                value = 5
                for move in poss_moves:
                    t_board = chess.Board(board.fen())
                    t_board.push(move)
                    t_value = ab_branch(t_board,n-1,eval,alpha,beta)[1]
                    if t_value < value:
                        value = t_value
                        best_move = move
                    elif t_value == value:
                        r = random.random()
                        if r > 0.1:
                            value = t_value
                            best_move = move
                    if value < alpha:
                        break
                    beta = min(beta,value) 
                    
                    
    else:
        if board.is_game_over():
            if board.is_checkmate():
                value = 2*(board.turn*(-2)+1)
                best_move = "checkmate?"
            elif board.is_insufficient_material():
                value = 0
                best_move = "insf material?"
            elif board.is_stalemate():
                value = 0
                best_move = "stalemate?"
        else:
            print(board.fen())
            
            
    return best_move,value